In [1]:
import pandas as pd
manf_df = pd.read_csv('Manufacturing_Facilities.csv')
logh_df = pd.read_csv('Logistics_Hubs.csv')


/tmp/ipykernel_4445/3610541987.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [4]:
print(manf_df)

   Facility_Id Facility_name Location  Capacity    Production_item  \
0            1       Manuf_A        A      5000  AtomicLayerEtcher   
1            2       Manuf_B        B      3000       PlasmaEtcher   
2            3       Manuf_C        C      7000       WaferCleaner   
3            4       Manuf_A        A      6000       PlasmaEtcher   
4            5       Manuf_D        D      2000  WaferCleanChamber   
5            6       Manuf_B        B      4500  AtomicLayerEtcher   

   Utilization_rate Maintenance_status  
0               0.8            Regular  
1               0.6               Good  
2               0.9               Good  
3               0.7           Critical  
4               0.8               Good  
5               0.5            Regular  


In [5]:
print(logh_df)

   Hub_Id Facility_Id Hub_name Location  Capacity  Throughput  \
0     101         1,2    Hub_X        X      8000         500   
1     102           2    Hub_Y        Y      6000         400   
2     103           3    Hub_Z        Z      9000         600   
3     104           4    Hub_A        A      7000         500   
4     105         5,6    Hub_B        B      5000         400   

   Inventory_Levels  
0              2000  
1              1500  
2              2500  
3              3000  
4              2000  


In [6]:

# Check for duplicate Facility id in the Manufact table (Primary Key constraint)
if manf_df.duplicated('Facility_Id').any():
    print("Primary key constraint violated in ManuFact table.")
else:
    print("No primary key constraint violation in ManuFact table.")

# Check for duplicate HubID in the LogHub table (Primary Key constraint)
if logh_df.duplicated('Hub_Id').any():
    print("Primary key constraint violated in LogHub table.")
else:
    print("No primary key constraint violation in LogHub table.")

# Check if all FacilityIDs in the LogHub table exist in the Manufact table (Foreign Key constraint)
if not manf_df['Facility_Id'].isin(logh_df['Hub_Id']).all():
    print("Foreign key constraint violated in LogHub table.")
else:
    print("No foreign key constraint violation in LogHub table.")

No primary key constraint violation in ManuFact table.
No primary key constraint violation in LogHub table.
Foreign key constraint violated in LogHub table.


In [13]:
# Concatenate DataFrames along rows axis (index)
concatenated_df = pd.concat([logh_df, manf_df])

print(concatenated_df)

   Hub_Id Facility_Id Hub_name Location  Capacity  Throughput  \
0   101.0         1,2    Hub_X        X      8000       500.0   
1   102.0           2    Hub_Y        Y      6000       400.0   
2   103.0           3    Hub_Z        Z      9000       600.0   
3   104.0           4    Hub_A        A      7000       500.0   
4   105.0         5,6    Hub_B        B      5000       400.0   
0     NaN           1      NaN        A      5000         NaN   
1     NaN           2      NaN        B      3000         NaN   
2     NaN           3      NaN        C      7000         NaN   
3     NaN           4      NaN        A      6000         NaN   
4     NaN           5      NaN        D      2000         NaN   
5     NaN           6      NaN        B      4500         NaN   

   Inventory_Levels Facility_name    Production_item  Utilization_rate  \
0            2000.0           NaN                NaN               NaN   
1            1500.0           NaN                NaN               NaN 

In [47]:
# Assuming 'primary_key_column' is the primary key column name and 'foreign_key_column' is the foreign key column name
primary_key_column = 'Hub_Id'
foreign_key_column = 'Facility_Id'

# Concatenate DataFrames along rows axis (index)
concatenated_df = pd.concat([logh_df, manf_df])
#print(concatenated_df)

# Group by primary key column
groups = concatenated_df.groupby(primary_key_column)

# Initialize an empty dictionary to store primary keys with multiple foreign keys
primary_multiple_foreign_keys = {}

# Iterate through groups
for primary_key, group_df in groups:
    foreign_keys = group_df[foreign_key_column].unique()
    #print(group_df)
    #print(foreign_keys)
    if len(foreign_keys) >=1:
        primary_multiple_foreign_keys[primary_key] = foreign_keys
        #print(primary_multiple_foreign_keys)

# Print primary keys with multiple foreign keys
for primary_key, foreign_keys in primary_multiple_foreign_keys.items():
    print(f"Primary key '{primary_key}' has multiple foreign keys: {', '.join(map(str, foreign_keys))}")
      

Primary key '101.0' has multiple foreign keys: 1,2
Primary key '102.0' has multiple foreign keys: 2
Primary key '103.0' has multiple foreign keys: 3
Primary key '104.0' has multiple foreign keys: 4
Primary key '105.0' has multiple foreign keys: 5,6


In [48]:
# Assuming 'primary_key_column' is the primary key column name and 'foreign_key_column' is the foreign key column name
primary_key_column = 'Hub_Id'
foreign_key_column = 'Facility_Id'

# Concatenate DataFrames along rows axis (index)
concatenated_df = pd.concat([logh_df, manf_df])
#print(concatenated_df)

# Group by primary key column
groups = concatenated_df.groupby(primary_key_column)

# Initialize an empty list to store primary keys with multiple foreign keys as well as single foreign keys
rows_foreign_keys = []
# Iterate through groups
for primary_key, group_df in groups:
    foreign_keys = group_df[foreign_key_column].unique()
    if len(foreign_keys) >= 1:
        rows_foreign_keys.append({'Hub_Id': primary_key, 'Facility_Id': ', '.join(map(str, foreign_keys))})

# Create a DataFrame from the list
df_multiple_foreign_keys = pd.DataFrame(rows_foreign_keys)

# Print the DataFrame containing primary keys with multiple foreign keys
print(df_multiple_foreign_keys)

   Hub_Id Facility_Id
0   101.0         1,2
1   102.0           2
2   103.0           3
3   104.0           4
4   105.0         5,6


In [39]:
for foreign_keys in df_multiple_foreign_keys['Foreign_Keys']:
    keys = foreign_keys.split(', ')
    for key in keys:
        print(key)

1,2
2
3
4
5,6


In [42]:
# Iterate through the DataFrame and print each primary key with each associated foreign key separately
for index, row in df_multiple_foreign_keys.iterrows():
    primary_key = row['Primary_Key']
    foreign_keys = [key.strip() for key in row['Foreign_Keys'].split(',')]
    for foreign_key in foreign_keys:
        print(f"Primary key '{primary_key}' is associated with foreign key '{foreign_key}'")

Primary key '101.0' is associated with foreign key '1'
Primary key '101.0' is associated with foreign key '2'
Primary key '102.0' is associated with foreign key '2'
Primary key '103.0' is associated with foreign key '3'
Primary key '104.0' is associated with foreign key '4'
Primary key '105.0' is associated with foreign key '5'
Primary key '105.0' is associated with foreign key '6'


In [45]:
import pandas as pd

# Initialize an empty list to store combinations of primary key and foreign key
combined_data = []

# Iterate through the DataFrame and collect combinations
for index, row in df_multiple_foreign_keys.iterrows():
    primary_key = row['Hub_Id']
    foreign_keys = [key.strip() for key in row['Facility_Id'].split(',')]
    for foreign_key in foreign_keys:
        combined_data.append({'Hub_Id': primary_key, 'Facility_Id': foreign_key})

# Convert the list of dictionaries into a DataFrame
combined_df = pd.DataFrame(combined_data)

# Print the DataFrame
print(combined_df)

   Hub_Id Facility_Id
0   101.0           1
1   101.0           2
2   102.0           2
3   103.0           3
4   104.0           4
5   105.0           5
6   105.0           6
